In [1]:
!pip install -q torch_geometric
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv, GATConv
import matplotlib.pyplot as plt
from copy import deepcopy
from tqdm.auto import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.3 MB/s eta 0:00:00


In [ ]:
# cites = '/content/Graph-Convolutional-Networks-with-Cora-Dataset/cora/cora.cites'
# content = '/content/Graph-Convolutional-Networks-with-Cora-Dataset/cora/cora.content'

# edge_df = pd.read_csv(cites, sep='\t', header=None, names=["target", "source"])
# column_names = ["w_{}".format(i) for i in range(1433)] + ["subject"]
# node_df = pd.read_csv(content, sep='\t', header=None, names=column_names)
# edge_data = edge_df.values.T
# node_data = node_df.values[:, :-1]
# labels = node_df.values[:, -1]

In [2]:
med_data = Planetoid(root='/tmp/PubMed', name='PubMed')[0]
cite_data = Planetoid(root='/tmp/CiteSeer', name='CiteSeer')[0]
cora_data = Planetoid(root='/tmp/Cora', name='Cora')[0]
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Processing...
Done!
Processing...
Done!
Processing...
Done!


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.nn.inits import glorot, zeros

class MultiGraphConvolution(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(MultiGraphConvolution, self).__init__(aggr='add')
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.weight1 = torch.nn.Parameter(torch.Tensor(in_channels, out_channels))
        self.weight2 = torch.nn.Parameter(torch.Tensor(in_channels, out_channels))
        self.att = torch.nn.Parameter(torch.Tensor(2))
        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight1)
        glorot(self.weight2)
        zeros(self.att)

    def forward(self, x, edge_index1, edge_index2):
        x = F.dropout(x, p=0.5, training=self.training)

        edge_index1, _ = add_self_loops(edge_index1, num_nodes=x.size(0))
        edge_index2, _ = add_self_loops(edge_index2, num_nodes=x.size(0))

        row1, col1 = edge_index1
        row2, col2 = edge_index2

        deg1 = degree(row1, x.size(0), dtype=x.dtype)
        deg2 = degree(row2, x.size(0), dtype=x.dtype)

        norm1 = deg1.pow(-0.5)
        norm2 = deg2.pow(-0.5)

        norm1 = norm1[row1] * norm1[col1]
        norm2 = norm2[row2] * norm2[col2]

        x1 = torch.matmul(x, self.weight1)
        x2 = torch.matmul(x, self.weight2)

        out1 = self.propagate(edge_index1, x=x1, norm=norm1)
        out2 = self.propagate(edge_index2, x=x2, norm=norm2)

        alpha = F.softmax(self.att, dim=0)
        out = alpha[0] * out1 + alpha[1] * out2

        return out

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        return aggr_out


In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class MAGCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(MAGCN, self).__init__()
        self.conv1 = MultiGraphConvolution(num_node_features, 16)
        self.conv2 = MultiGraphConvolution(16, num_classes)

    def forward(self, x, edge_index1, edge_index2):
        x = self.conv1(x, edge_index1, edge_index2)
        x = F.relu(x)
        x = self.conv2(x, edge_index1, edge_index2)
        return F.log_softmax(x, dim=1)


In [ ]:
class MultiGraphConvolution(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(MultiGraphConvolution, self).__init__(aggr='add')
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.weight1 = torch.nn.Parameter(torch.Tensor(in_channels, out_channels))
        self.weight2 = torch.nn.Parameter(torch.Tensor(in_channels, out_channels))
        self.att = torch.nn.Parameter(torch.Tensor(2))
        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight1)
        glorot(self.weight2)
        zeros(self.att)

    def forward(self, x, edge_index1, edge_index2):
        x = F.dropout(x, p=0.5, training=self.training)

        edge_index1, _ = add_self_loops(edge_index1, num_nodes=x.size(0))
        edge_index2, _ = add_self_loops(edge_index2, num_nodes=x.size(0))

        row1, col1 = edge_index1
        row2, col2 = edge_index2

        deg1 = degree(row1, x.size(0), dtype=x.dtype)
        deg2 = degree(row2, x.size(0), dtype=x.dtype)

        norm1 = deg1.pow(-0.5)
        norm2 = deg2.pow(-0.5)

        norm1 = norm1[row1] * norm1[col1]
        norm2 = norm2[row2] * norm2[col2]

        x1 = torch.matmul(x, self.weight1)
        x2 = torch.matmul(x, self.weight2)

        out1 = self.propagate(edge_index1, x=x1, norm=norm1)
        out2 = self.propagate(edge_index2, x=x2, norm=norm2)

        alpha = F.softmax(self.att, dim=0)
        out = alpha[0] * out1 + alpha[1] * out2

        return out

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        return aggr_out

class MAGCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(MAGCN, self).__init__()
        self.conv1 = MultiGraphConvolution(num_node_features, 16)
        self.conv2 = MultiGraphConvolution(16, num_classes)

    def forward(self, x, edge_index1, edge_index2):
        x = self.conv1(x, edge_index1, edge_index2)
        x = F.relu(x)
        x = self.conv2(x, edge_index1, edge_index2)
        return F.log_softmax(x, dim=1)

In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Data

# Load Cora dataset
dataset = Planetoid(root='/tmp/PubMed', name='PubMed', transform=NormalizeFeatures())
data = dataset[0]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MAGCN(dataset.num_node_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_index)  # Assuming single edge_index for simplicity
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    logits = model(data.x, data.edge_index, data.edge_index)  # Assuming single edge_index for simplicity
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

for epoch in range(2000):
    loss = train()
    train_acc, val_acc, test_acc = test()
    print(f'Epoch: {epoch+1:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')


Processing...
Done!


Epoch: 001, Loss: 1.0981, Train Acc: 0.4333, Val Acc: 0.3700, Test Acc: 0.3530
Epoch: 002, Loss: 1.0982, Train Acc: 0.4833, Val Acc: 0.3840, Test Acc: 0.3710
Epoch: 003, Loss: 1.0984, Train Acc: 0.5000, Val Acc: 0.3960, Test Acc: 0.3850
Epoch: 004, Loss: 1.0980, Train Acc: 0.5500, Val Acc: 0.4080, Test Acc: 0.4030
Epoch: 005, Loss: 1.0978, Train Acc: 0.6000, Val Acc: 0.4240, Test Acc: 0.4120
Epoch: 006, Loss: 1.0977, Train Acc: 0.6000, Val Acc: 0.4300, Test Acc: 0.4200
Epoch: 007, Loss: 1.0970, Train Acc: 0.6167, Val Acc: 0.4380, Test Acc: 0.4310
Epoch: 008, Loss: 1.0968, Train Acc: 0.6167, Val Acc: 0.4440, Test Acc: 0.4370
Epoch: 009, Loss: 1.0967, Train Acc: 0.6167, Val Acc: 0.4500, Test Acc: 0.4440
Epoch: 010, Loss: 1.0962, Train Acc: 0.6167, Val Acc: 0.4620, Test Acc: 0.4510
Epoch: 011, Loss: 1.0964, Train Acc: 0.6167, Val Acc: 0.4760, Test Acc: 0.4600
Epoch: 012, Loss: 1.0961, Train Acc: 0.6333, Val Acc: 0.4800, Test Acc: 0.4610
Epoch: 013, Loss: 1.0959, Train Acc: 0.6333, Val Acc

# new combined model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, GCNConv
from torch_geometric.utils import add_self_loops, degree

# MultiGraphConvolution class from the second code
class MultiGraphConvolution(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(MultiGraphConvolution, self).__init__(aggr='add')
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.weight1 = torch.nn.Parameter(torch.Tensor(in_channels, out_channels))
        self.weight2 = torch.nn.Parameter(torch.Tensor(in_channels, out_channels))
        self.att = torch.nn.Parameter(torch.Tensor(2))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight1)
        nn.init.xavier_uniform_(self.weight2)
        nn.init.zeros_(self.att)

    def forward(self, x, edge_index1, edge_index2):
        x = F.dropout(x, p=0.5, training=self.training)

        edge_index1, _ = add_self_loops(edge_index1, num_nodes=x.size(0))
        edge_index2, _ = add_self_loops(edge_index2, num_nodes=x.size(0))

        row1, col1 = edge_index1
        row2, col2 = edge_index2

        deg1 = degree(row1, x.size(0), dtype=x.dtype)
        deg2 = degree(row2, x.size(0), dtype=x.dtype)

        norm1 = deg1.pow(-0.5)
        norm2 = deg2.pow(-0.5)

        norm1 = norm1[row1] * norm1[col1]
        norm2 = norm2[row2] * norm2[col2]

        x1 = torch.matmul(x, self.weight1)
        x2 = torch.matmul(x, self.weight2)

        out1 = self.propagate(edge_index1, x=x1, norm=norm1)
        out2 = self.propagate(edge_index2, x=x2, norm=norm2)

        alpha = F.softmax(self.att, dim=0)
        out = alpha[0] * out1 + alpha[1] * out2

        return out

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        return aggr_out

# Attention mechanism adapted for use with the MultiGraphConvolution
class Attention1(nn.Module):
    def __init__(self, n_view, in_ch, out_ch):
        super(Attention1, self).__init__()
        self.n_view = n_view
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.mlp = nn.Sequential(
            nn.Linear(n_view, 6), nn.ReLU(),
            nn.Linear(6, 3), nn.ReLU(),
            nn.Linear(3, n_view)
        )
        self.fc = nn.Linear(in_ch, in_ch)

    def forward(self, x):
        new_x = torch.stack(x)
        new_x = self.gap(new_x).permute((1, 2, 0))
        a = F.softmax(self.mlp(new_x), dim=-1).permute((2, 0, 1))
        weighted_x = torch.sum(a * torch.stack(x), dim=0)
        weighted_x = self.fc(weighted_x)
        return weighted_x

# Combined MAGCN model with attention
class MAGCNWithAttention(nn.Module):
    def __init__(self, num_node_features, num_classes, n_view=2, hidden_dim=16):
        super(MAGCNWithAttention, self).__init__()
        self.n_view = n_view
        self.conv_layers = nn.ModuleList([MultiGraphConvolution(num_node_features, hidden_dim) for _ in range(n_view)])
        self.attention = Attention1(n_view, hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, edge_index1, edge_index2):
        # Apply multiple GCNs and collect outputs
        gcn_outputs = [conv_layer(x, edge_index1, edge_index2) for conv_layer in self.conv_layers]
        # Apply attention mechanism
        x = self.attention(gcn_outputs)
        x = F.relu(x)
        # Final classification layer
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
from torch_geometric.utils import train_test_split_edges

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Prepare the data (Cora dataset only has one graph)
data = dataset[0]

# Split the edges into train/val/test sets
data = train_test_split_edges(data)

# Print some statistics
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Number of node features: {data.num_node_features}')
print(f'Number of classes: {dataset.num_classes}')


# Define the model
model = MAGCNWithAttention(num_node_features=data.num_node_features,
                           num_classes=dataset.num_classes)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
data = data.to(device)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.train_pos_edge_index, data.train_pos_edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test():
    model.eval()
    out = model(data.x, data.train_pos_edge_index, data.train_pos_edge_index)
    pred = out.argmax(dim=1)

    # Accuracy calculation
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        accs.append(correct / mask.sum().item())
    return accs

# Training loop
epochs = 2000
for epoch in range(epochs):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 10 == 0:  # Print every 10 epochs
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, '
              f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

train_acc, val_acc, test_acc = test()
print(f'Final Test Accuracy: {test_acc:.4f}')


Number of nodes: 2708
Number of edges: 263
Number of node features: 1433
Number of classes: 7
Epoch 000, Loss: 1.9590, Train Acc: 0.1429, Val Acc: 0.1560, Test Acc: 0.1440
Epoch 010, Loss: 1.2252, Train Acc: 0.9286, Val Acc: 0.5080, Test Acc: 0.5460
Epoch 020, Loss: 0.2865, Train Acc: 0.9929, Val Acc: 0.6740, Test Acc: 0.7130
Epoch 030, Loss: 0.0397, Train Acc: 1.0000, Val Acc: 0.7160, Test Acc: 0.7250
Epoch 040, Loss: 0.0161, Train Acc: 1.0000, Val Acc: 0.7100, Test Acc: 0.7350
Epoch 050, Loss: 0.0161, Train Acc: 1.0000, Val Acc: 0.7140, Test Acc: 0.7360
Epoch 060, Loss: 0.0169, Train Acc: 1.0000, Val Acc: 0.7240, Test Acc: 0.7290
Epoch 070, Loss: 0.0163, Train Acc: 1.0000, Val Acc: 0.7180, Test Acc: 0.7440
Epoch 080, Loss: 0.0093, Train Acc: 1.0000, Val Acc: 0.7080, Test Acc: 0.7470
Epoch 090, Loss: 0.0162, Train Acc: 1.0000, Val Acc: 0.7200, Test Acc: 0.7550
Epoch 100, Loss: 0.0118, Train Acc: 1.0000, Val Acc: 0.7360, Test Acc: 0.7630
Epoch 110, Loss: 0.0094, Train Acc: 1.0000, Val 